In [12]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 

Train form data in dataset using data augmentation

In [5]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(3,'dataset','images','labels',data_gen_args,save_to_dir = None)
model = satellite_unet()
model_checkpoint = ModelCheckpoint('sat_unet.hdf5', monitor='acc',verbose=1, save_best_only=True)
model.fit_generator(train_generator,steps_per_epoch=800,epochs=5,validation_data=val_generator,validation_steps=120, callbacks=[model_checkpoint])

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 800 steps, validate for 120 steps
Epoch 1/5
800/800 [==============================] - 118s 147ms/step - loss: 0.3086 - accuracy: 0.7735 - mean_io_u: 0.3872 - val_loss: 2.4169 - val_accuracy: 0.3749 - val_mean_io_u: 0.3397
Epoch 2/5
800/800 [==============================] - 106s 133ms/step - loss: 0.2135 - accuracy: 0.8110 - mean_io_u: 0.3872 - val_loss: 0.8601 - val_accuracy: 0.5874 - val_mean_io_u: 0.3405
Epoch 3/5
800/800 [==============================] - 107s 133ms/step - loss: 0.1623 - accuracy: 0.8261 - mean_io_u: 0.3871 - val_loss: 0.9847 - val_accuracy: 0.5671 - val_mean_io_u: 0.3404
Epoch 4/5
800/800 [==============================] - 107s 134ms/step - loss: 0.1337 - accuracy: 0.8337 - 

Test model

In [ ]:
model=satellite_unet(pretrained='sat_unet.hdf5')
X_test,names=test("dataset/unlabelled")
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults("dataset/preds",results,names)

In [24]:
def test(test_path,target_size=(256,256), color_mode = "rgb"):
    X_test=[]
    names=[]
    for filename in os.listdir(test_path):
        name, ext = os.path.splitext(filename)
        if ext!=".png" and ext!=".jpg":
            continue
        names.append(filename)
        img=load_img(os.path.join(test_path,filename),target_size=(256,256), color_mode = color_mode)
        img=img_to_array(img)/255
        X_test.append(img.copy())
    X_test_label = np.array(X_test)
    return X_test_label

test_img = test('dataset/images_satelite')
# test_img = test_img[1:,:,:,:]
print(test_img.shape)

test_label = test('dataset/labels_satelite', color_mode = "grayscale")
print(test_label.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/images_satelite'

In [46]:
model=satellite_unet(pretrained='sat_unet.hdf5')

score, acc = model.evaluate(test_img, test_label)
print('Test score:', score)
print('Test accuracy:', acc)

18/18 [==============================] - 1s 52ms/sample - loss: 1.5818 - accuracy: 0.6035
Test score: 1.581809163093567
Test accuracy: 0.6035216


In [49]:
# data_gen_args = dict(rotation_range=0.2,
#                     width_shift_range=0.05,
#                     height_shift_range=0.05,
#                     shear_range=0.05,
#                     zoom_range=0.05,
#                     horizontal_flip=True,
#                     vertical_flip=True,
#                     fill_mode='nearest')
# myGene = trainGenerator(3,'dataset','images','labels',data_gen_args,save_to_dir = None)
# model = satellite_unet()
# model_checkpoint = ModelCheckpoint('sat_unet_IoU.hdf5', monitor='acc',verbose=1, save_best_only=False)
# model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

In [15]:
# model=satellite_unet(pretrained='sat_unet_IoU.hdf5')

# score, acc = model.evaluate(test_img, test_label)
# print('Test score:', score)
# print('Test accuracy:', acc)

18/18 [==============================] - 1s 60ms/sample - loss: 1.9629 - mean_io_u_1: 0.4234
Test score: 1.9629113674163818
Test accuracy: 0.4234361


In [2]:
# Tried data from the same distribution, split data from our satelite images into train and validation sets

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict=data_gen_args,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    mask_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'training',
        seed = seed) 
    image_val_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'validation',
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'training',
        seed = seed)
    mask_val_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'validation',
        seed = seed)
    
        
    return image_generator, image_val_generator, mask_generator, mask_val_generator


In [3]:
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(3,'dataset','images_satellite','labels_satellite',data_gen_args)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [4]:
def train(image_generator, mask_generator, flag_multi_class = False, num_class = 2):
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)       
        yield (img,mask)
        
def val(image_val_generator, mask_val_generator, flag_multi_class = False, num_class = 2):
    val_generator = zip(image_val_generator, mask_val_generator)
    for (img_val,mask_val) in val_generator:
        img_val,mask_val = adjustData(img_val,mask_val,flag_multi_class,num_class)       
        yield (img_val,mask_val)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

In [21]:
from tensorflow.keras import backend as K

def Mean_IOU_tensorflow_2(y_true, y_pred):
    nb_classes = K.int_shape(y_pred)[-1]
    iou = []
    true_pixels = K.argmax(y_true, axis=-1)
    pred_pixels = K.argmax(y_pred, axis=-1)
    void_labels = K.equal(K.sum(y_true, axis=-1), 0)
    for i in range(0, nb_classes): # exclude first label (background) and last label (void)
        true_labels = K.equal(true_pixels, i) & ~void_labels
        pred_labels = K.equal(pred_pixels, i) & ~void_labels
        inter = tf.dtypes.cast(true_labels & pred_labels, tf.int32)
        union = tf.dtypes.cast(true_labels | pred_labels, tf.int32)
        legal_batches = K.sum(tf.dtypes.cast(true_labels, tf.int32), axis=1)>0
        ious = K.sum(inter, axis=1)/K.sum(union, axis=1)
        iou.append(K.mean(ious[legal_batches]))
    iou = tf.stack(iou)
    legal_labels = ~tf.math.is_nan(iou)
    iou = iou[legal_labels]
    return K.mean(iou)

def sparse_Mean_IOU(y_true, y_pred):
    nb_classes = K.int_shape(y_pred)[-1]
    iou = []
    pred_pixels = K.argmax(y_pred, axis=-1)
    for i in range(0, nb_classes): # exclude first label (background) and last label (void)
        true_labels = K.equal(y_true[:,:,0], i)
        pred_labels = K.equal(pred_pixels, i)
        inter = tf.dtypes.cast(true_labels & pred_labels, tf.int32)
        union = tf.dtypes.cast(true_labels | pred_labels, tf.int32)
        legal_batches = K.sum(tf.dtypes.cast(true_labels, tf.int32), axis=1)>0
        ious = K.sum(inter, axis=1)/K.sum(union, axis=1)
        iou.append(K.mean(tf.gather(ious, indices=tf.where(legal_batches)))) # returns average IoU of the same objects
    iou = tf.stack(iou)
    legal_labels = ~tf.math.is_nan(iou)
    iou = tf.gather(iou, indices=tf.where(legal_labels))
    return K.mean(iou)

model = satellite_unet()
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', sparse_Mean_IOU])

In [22]:
model_checkpoint = ModelCheckpoint('sat_unet_new.hdf5', monitor='val_accuracy',verbose=1, save_best_only=True)
model.fit_generator(train_generator,steps_per_epoch=100,epochs=15,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/15
 99/100 [============================>.] - ETA: 0s - loss: 0.4750 - accuracy: 0.6887 - sparse_Mean_IOU: 0.7608
Epoch 00001: val_accuracy improved from -inf to 0.67993, saving model to sat_unet_new.hdf5
100/100 [==============================] - 20s 198ms/step - loss: 0.4735 - accuracy: 0.6887 - sparse_Mean_IOU: 0.7592 - val_loss: 0.4562 - val_accuracy: 0.6799 - val_sparse_Mean_IOU: 0.5677
Epoch 2/15
 99/100 [============================>.] - ETA: 0s - loss: 0.3822 - accuracy: 0.7389 - sparse_Mean_IOU: 0.7805
Epoch 00002: val_accuracy improved from 0.67993 to 0.73964, saving model to sat_unet_new.hdf5
100/100 [==============================] - 16s 157ms/step - loss: 0.3827 - accuracy: 0.7388 - sparse_Mean_IOU: 0.7806 - val_loss: 0.3750 - val_accuracy: 0.7396 - val_sparse_Mean_IOU: 0.5513
Epoch 3/15
 99/100 [============================>.] - ETA: 0s - loss: 0.3636 - accuracy: 0.7412 - 

In [25]:
test_img = test('dataset/test_images')
test_label = test('dataset/test_labels', color_mode = "grayscale")
model=satellite_unet(pretrained='sat_unet_new.hdf5')

score= model.evaluate(test_img, test_label)
print('Test score:', score)
# print('Test accuracy:', acc)
# print('Test IoU:', Iou)

ValueError: Unknown metric function:sparse_Mean_IOU

In [1]:
def test2(test_path,target_size=(256,256)):
    X_test=[]
    names=[]
    for filename in os.listdir(test_path):
        name, ext = os.path.splitext(filename)
        if ext!=".png" and ext!=".jpg":
            continue
        names.append(filename)
        img=load_img(os.path.join(test_path,filename),target_size=target_size)
        img=img_to_array(img)/255
        X_test.append(img.copy())
    return np.array(X_test),names
    

In [26]:
### TO DO ####
import os

model=satellite_unet(pretrained='sat_unet_new.hdf5')
X_test,names=test2("dataset/images")
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults(os.getcwd()+"/dataset/predict",results,names,empty_dir=False)

ValueError: Unknown metric function:sparse_Mean_IOU